In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [3]:

conf = SparkConf()  # create the configuration
# conf.set("spark.jars", "/home/serendipita/Documents/utec/bigdata-2022-2/Lab09-spark/notebooks/postgresql-42.4.0.jar")  # set the spark.jars

spark = SparkSession.builder.config(conf=conf).master("local").appName("Python Spark SQL basic example").getOrCreate()



In [4]:

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/sparksql?user=sparksql") \
    .option("dbtable", "gameclicks") \
    .load() # \
    #.show(10)

# df.printSchema()

In [6]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- clickid: integer (nullable = true)
 |-- userid: integer (nullable = true)
 |-- usersessionid: integer (nullable = true)
 |-- ishit: integer (nullable = true)
 |-- teamid: integer (nullable = true)
 |-- teamlevel: integer (nullable = true)



In [7]:
df.count()

755806

In [8]:
df.show()

+-------------------+-------+------+-------------+-----+------+---------+
|          timestamp|clickid|userid|usersessionid|ishit|teamid|teamlevel|
+-------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:55|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:09|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:14|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:14|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:20|     22|  1362|         5739|    0|    13|        1|
|2016-05-26 15:07:27|    107|  1071|         5939|    0|    27|        1|
|2016-05-26 15:07:30|    289|  2359|         5764|    0|    85|        1|
|2016-05-26 15:07:30|    301|  1243|         5900|    0|    86|        1|
|2016-05-26 15:07:47|    274|  1628|         5896|    0|    82|        1|
|2016-05-26 15:07:48|     66|   453|         5662|    0|    20|        1|
|2016-05-26 15:07:49|    124|  2336|  

In [9]:
df.select('userid', 'teamlevel').show()

+------+---------+
|userid|teamlevel|
+------+---------+
|  1038|        1|
|  1099|        1|
|   899|        1|
|  2197|        1|
|  1362|        1|
|  1071|        1|
|  2359|        1|
|  1243|        1|
|  1628|        1|
|   453|        1|
|  2336|        1|
|  1664|        1|
|   243|        1|
|  2333|        1|
|   181|        1|
|   212|        1|
|  1142|        1|
|  1085|        1|
|  1215|        1|
|  2306|        1|
+------+---------+
only showing top 20 rows



In [14]:
df.filter(df['teamlevel']>7).select('userid', 'teamlevel').show()

+------+---------+
|userid|teamlevel|
+------+---------+
|  1148|        8|
|  1806|        8|
|   765|        8|
|  1966|        8|
|  1769|        8|
|  2187|        8|
|   960|        8|
|  1423|        8|
|  1373|        8|
|  1742|        8|
|   720|        8|
|   464|        8|
|   591|        8|
|  1256|        8|
|   511|        8|
|   920|        8|
|   371|        8|
|  1966|        8|
|  1797|        8|
|  1756|        8|
+------+---------+
only showing top 20 rows



In [15]:
df.groupby('ishit').count().show()

+-----+------+
|ishit| count|
+-----+------+
|    1| 83383|
|    0|672423|
+-----+------+



In [16]:
from pyspark.sql.functions import *

In [17]:
df.select(mean('ishit'), sum('ishit')).show()

+------------------+----------+
|        avg(ishit)|sum(ishit)|
+------------------+----------+
|0.1103232840173272|     83383|
+------------------+----------+



In [18]:

df2 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/sparksql?user=sparksql") \
    .option("dbtable", "adclicks") \
    .load() #

In [19]:
merge = df.join(df2, 'userid')

In [20]:
merge.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- clickid: integer (nullable = true)
 |-- usersessionid: integer (nullable = true)
 |-- ishit: integer (nullable = true)
 |-- teamid: integer (nullable = true)
 |-- teamlevel: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- txid: integer (nullable = true)
 |-- usersessionid: integer (nullable = true)
 |-- teamid: integer (nullable = true)
 |-- adid: integer (nullable = true)
 |-- adcategory: string (nullable = true)



In [21]:
merge.show()

+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+----------+
|userid|          timestamp|clickid|usersessionid|ishit|teamid|teamlevel|          timestamp| txid|usersessionid|teamid|adid|adcategory|
+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+----------+
|   471|2016-05-26 15:12:28|    109|         5863|    0|    27|        1|2016-06-15 07:40:58|37570|        34330|    27|   6|    movies|
|   471|2016-05-26 15:12:28|    109|         5863|    0|    27|        1|2016-06-15 00:10:58|37103|        34330|    27|  23|   fashion|
|   471|2016-05-26 15:12:28|    109|         5863|    0|    27|        1|2016-06-14 13:06:53|36168|        34330|    27|  26|    movies|
|   471|2016-05-26 15:12:28|    109|         5863|    0|    27|        1|2016-06-14 09:31:41|35833|        34330|    27|  12| computers|
|   471|2016-05-26 15:12:28|    109|     